<a href="https://colab.research.google.com/github/s-eee/KeibaAI/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import sys

# スクレイピングコード非公開のため
sys.path.append("/content/drive/MyDrive/競馬(公開)/プログラム")
import scraping as scr

In [ ]:
# 元データと同様に標準化
def z_score(X):
    mean = 34.404076639868435
    std = 164.14767907079175
    X -= mean
    X /= std
    return X

In [ ]:
# 1レースを予想
def predict(model_pass, url, num, skip_num=None):
    model = keras.models.load_model(model_pass, compile=False)
    # データをスクレイピング
    data = scr.scrape(url, num=num, skip_num=skip_num)
    # 整形
    X = scr.data_formating(data)
    # 標準化
    X = z_score(X)
    Y = {}
    keys = []
    values = []

    print(f"第{data[0][0]}レース")
    # 10頭を予想
    for i in range(10):
        x = X[i]
        x = x.reshape(1, 43)
        y = model.predict(x)
        keys.append(str(i+1) + data[i][12])
        values.append(y)
        print(f"馬番{i + 1}({data[i][12]}) : {y}")
    
    # 降順に並べ替えてプリント
    Y.update(zip(keys, values))
    Y = sorted(Y.items(), key=lambda x:x[1], reverse=True)
    print("-"*10 +"昇順" + "-"*10)
    for key, value in Y:
        print(key,value)

    return Y

In [ ]:
import re

# 1日のレース予想と結果を検証
def verification(url, model_pass, skip_num=None,skip_num2=None):
    # 複勝当選回数
    score = 0
    # 複勝
    first_score = 0
    second_score = 0
    third_score = 0
    # ワイド
    wide = 0
    # 三連複
    sanren = 0
    try:
        for race_num in range(12):
          # 引数skip_numのレースをスキップ
            if race_num == skip_num:
                continue
            if race_num == skip_num2:
                continue
            # 予想
            Y = predict(model_pass, url, race_num)
            # 結果をスクレイピング
            T = scr.get_answer(url, race_num)
            # カウント
            c = 0
            # 予想上位３頭が３着以内に入っている回数をカウント
            for i in range(3):
                y = re.sub(r"\D", "", Y[i][0])
                if int(y) == int(T[0]) or int(y) == int(T[1]) or int(y) == int(T[2]):
                    score += 1
                    c += 1
                    if i == 0:
                        first_score += 1
                    if i == 1:
                        second_score += 1
                    if i == 2:
                        third_score += 1
            # 予想上位３頭の中で２頭以上当選していたらワイド＋１
            if c >= 2:
                wide += 1
                print("ワイド的中")
            # 予想上位３頭の中で３頭当選していたら三連複＋１
            if c == 3:
                sanren += 1
                print("三連的中")
            print(f"複勝当選数（予想1番目）　：　{first_score}")
            print(f"複勝当選数（予想2番目）　：　{second_score}")
            print(f"複勝当選数（予想3番目）　：　{third_score}")
            print(f"複勝当選馬数(合計)　：　{score}")
            print(f"ワイド　当選回数　：　{wide}")
            print(f"3連複 当選回数　：　{sanren}")

    except IndexError:
      # １２レース未満 or まだレースがスタートしていなければIndexError
        print("indexError")
        return Y,T
    else:
        return Y,T


In [ ]:
# 新レース検証
model_list = [""]
url = ""

for model in model_list:
    print(model)
    Y,T = verification(url, model)


In [ ]:
# 新モデル検証
url_list = [""]
model_pass = ""

for url in url_list:
    print(url)
    Y,T = verification(url, model_pass)

In [ ]:
# 1レース予想
model_pass = ""
url = ""


Y = predict(model_pass, url, )